In [1]:
from twisted.internet import protocol, reactor, endpoints
from twisted.internet.protocol import Factory, connectionDone
from twisted.protocols import basic
from twisted.protocols import wire
import time
import logging
import configparser
import pika
from vsail_data_parser import VsailDataParser

In [2]:
#Vsail车辆数据接收服务
class VsailServer(object):
    def __init__(self):
        logging.basicConfig(filename="twisted_server.log", filemode="w", format="%(asctime)s %(name)s:%(levelname)s:%(message)s", datefmt="%Y-%M-%d %H:%M:%S", level=logging.INFO)
        parser = configparser.ConfigParser()
        parser.read('application.ini')
        logging.info('加载application.ini配置')

        #获取rabbitmq配置信息
        rq_host = parser.get('rabbitmq', 'host')
        #实时消息使用队列名称
        self.rq_real_ex_name = parser.get('rabbitmq', 'rq_real_ex_name')
        #历史消息使用队列名称 TODO暂不处理历史消息
        self.rq_his_ex_name =  parser.get('rabbitmq', 'rq_his_ex_name')

        #获取socket配置信息
        self.sk_host = parser.get('socket', 'host')
        self.sk_port = int(parser.get('socket', 'port'))
        self.init_rq = False
        self.rq_conn = self.init_rabbitmq(rq_host)
        
    #初始化rabbitMQ
    def init_rabbitmq(self, rq_host):
        try:
            parameters = pika.ConnectionParameters(rq_host, credentials=pika.credentials.PlainCredentials('admin','admin'), heartbeat=0)
            connection = pika.BlockingConnection(parameters)
            logging.info('rabbitmq已连接')
            self.init_rq = True
            return connection
        except Exception as ex:
            logging.exception('rabbitmq初始化错误')
            #异常出错不再往下执行
            raise Exception('rabbitmq初始化错误')
            
    def start(self):
        if self.init_rq:
            self.rt = reactor
            endpoints.serverFromString(reactor, "tcp:9999").listen(VsailDataFactory(self.rq_conn, self.rq_real_ex_name))
            reactor.run()
            
    def stop(self):
        if self.init_rq:
            self.rt.stop()
            self.rq_conn.close()

In [3]:
class VsailDataFactory(protocol.Factory):
    def __init__(self, rq_conn, ex_name):
        self.rq_conn = rq_conn
        self.ex_name = ex_name
    def buildProtocol(self, addr):
        channel = self.rq_conn.channel()
        print('创建channel')
        return VsailDataHandler(channel, self.ex_name)

In [4]:
class VsailDataHandler(basic.NetstringReceiver):
    def __init__(self, channel, ex_name):
        self.channel = channel
        self.ex_name = ex_name
        print('创建消息处理器')
    def dataReceived(self, data):
        print('开始接收消息...')
        message = data.decode('utf-8',"ignore")
        print(message)
        if message == 'exit':
            self.transport.loseConnection()
        else:
            try:
                parser = VsailDataParser(message)
                bus_data = parser.translate_to_json()
                #self.channel.basic_publish(exchange=self.ex_name, routing_key='', body=str(bus_data))
                self.channel.basic_publish(exchange=self.ex_name, routing_key='', body=message)
                #如果是上线或下线 返回结果指令信息
                if bus_data['type'] == 1 or bus_data['type'] == 2:
                    self.transport.write('OK'.encode('utf-8'))
                #else:
                    #self.transport.write('OKK'.encode('utf-8'))
            except Exception as ex:
                self.transport.write('error'.encode('utf-8'))
                #print(ex)
    def connectionMade(self):  # 建立连接后的回调函数
        #logging.info('客户端已连接')
        print('客户端已连接')
    def connectionLost(self, reason=connectionDone):  # 断开连接后的反应
        #logging.info('客户端已断开')
        print('客户端已断开')
        self.channel.close()
        

In [ ]:
if __name__ == '__main__':
    vsail_server = VsailServer()
    vsail_server.start()

In [ ]:
#vsail_server.stop()

In [ ]:
reactor.stop()